In [ ]:
import json, requests
import pandas as pd
import time

# Foursquare

In [ ]:
# api settings
client_id = os.environ['']
client_secret = os.environ['']
v = '20180323'

# getting restaurants in the neighborhood (NYC)
location = '40.7243,-74.0018'
url = 'https://api.foursquare.com/v2/venues/explore'

params = {
    'client_id': client_id,
    'client_secret': client_secret,
    'v': v,
    'll': location,
    'query': 'restaurant',
    'limit': 50,
    'radius': 1000
}

resp = requests.get(url=url, params=params)
data = resp.json()

In [ ]:
# restaurant details (rating,name,place, ...)

all_restaurants = []
for restaurant in data['response']['groups'][0]['items']:
    id_restaurant = restaurant['venue']['id']
    
    url = f'https://api.foursquare.com/v2/venues/{id_restaurant}'
    params = {
        'client_id': client_id,
        'client_secret': client_secret,
        'v': v
    }
    
    resp = requests.get(url=url, params=params)
    
    if resp.status_code == 200:
        restaurant_data = resp.json()
        venue = restaurant_data['response']['venue']

        all_restaurants.append(
            {
            'name': venue['name'],
            'address': ', '.join(venue['location']['formattedAddress']),
            'url': venue['url'],
            'price_category': venue['price']['tier'] if venue.get('price') else None,
            'likes': venue['likes']['count'],
            'rating': venue['rating'],
            'api': 'foursquare',
        })

In [ ]:
df_fs = pd.DataFrame(all_restaurants)

In [ ]:
df_fs

# Yelp

In [ ]:
# api settings
apikey = os.environ['']

headers = {
        'Authorization': f'Bearer {apikey}'
    }

# # getting restaurants in the neighborhood (NYC)
url_params = {
        'term': term.replace(' ', '+'),
        'latitude': '40.7243',
        'longitude': '-74.0018',
        'term': 'restaurants',
        'limit': 50,
        'radius': 1000
    }

url = 'https://api.yelp.com/v3/businesses/search'

resp = requests.get(url=url, headers=headers, params=url_params)
data = resp.json()

In [ ]:
# # restaurant details (rating,name,place, ...)

all_restaurants = []
for restaurant in data['businesses']:
    id_restaurant = restaurant['id']
    
    url = f'https://api.yelp.com/v3/businesses/{id_restaurant}'
    resp = requests.get(url=url, headers=headers)
    
    if resp.status_code == 200:
        restaurant_data = resp.json()
        
        all_restaurants.append(
            {
            'name': restaurant_data['name'],
            'address': ', '.join(restaurant_data['location']['display_address']),
            'url': restaurant_data['url'],
            'price_category': restaurant_data['price'].strip() if restaurant_data.get('price') else None,
            'likes': None,
            'rating': restaurant_data['rating']*2,
            'api': 'yelp',
        })

In [ ]:
df_yelp = pd.DataFrame(all_restaurants)
df_yelp['price_category'] = df_yelp['price_category'].apply(lambda x: len(x.strip()) if isinstance(x,str) else None)

In [ ]:
df_yelp

# Google 

In [ ]:
key = os.environ['']
radius = 1000
query = 'restaurant'
location = '40.7243,-74.0018'

In [ ]:
resp = requests.get(f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={location}&radius={radius}&type={query}&key={key}')
data = r.json()

results = []
results += data['results']
next_page_token = data.get('next_page_token')

while next_page_token:
    resp = requests.get(f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?pagetoken={next_page_token}&key={key}')
    data = resp.json()
    
    results += data['results']
    next_page_token = data.get('next_page_token')

In [ ]:
all_restaurants = []
for restaurant in results:
    all_restaurants.append(
            {
            'name': restaurant['name'],
            'address': restaurant['vicinity'],
            'url': None,
            'price_category': restaurant['price_level'],
            'likes': None,
            'rating': restaurant['rating']*2,
            'api': 'google',
        })
    

In [ ]:
df_google = pd.DataFrame(all_restaurants)

In [ ]:
df_google.head()

# Database

In [ ]:
import sqlite3

In [ ]:
# create connection
conn = sqlite3.connect("restaurants.db")

In [ ]:
# insert into db
df_fs.to_sql("restaurant_info", conn, if_exists="append", index=False)
df_yelp.to_sql("restaurant_info", conn, if_exists="append", index=False)
df_google.to_sql("restaurant_info", conn, if_exists="append", index=False)

In [ ]:
# all data from db
df_all = pd.read_sql('Select * from restaurant_info', conn)

In [ ]:
# top 10 restaurants according to rating
df_all.sort_values('rating',ascending=False).head(10)